# OpenQA Job Template Helper

This notebook helps with doing bulk operations on OpenQA job templates such as copying or moving the contents of a job group.

In [1]:
include("../src/init.jl")

using JDP.Trackers.OpenQA

┌ Info: Activating JDP package at /home/rich/julia/jdp/
└ @ Main /home/rich/julia/jdp/src/init.jl:5
┌ Info: Precompiling JDP [6d7e372e-c0bd-11e8-2b07-216b8359d694]
└ @ Base loading.jl:1186


## Product defs map

Create a product definition map for the different architectures. Make sure to set the desired product version.

In [ ]:
prods = OpenQA.get_json(OpenQA.osd, "products")["Products"]
prods[1]

In [ ]:
aprods = map(filter(prods) do p
        p["flavor"] == "Installer-DVD" && p["distri"] == "sle" && p["version"] == "15-SP1"
    end) do p
        p["arch"] => p["id"]
    end |> Dict

In [3]:
jobids = OpenQA.get_json(OpenQA.osd, "jobs?group_id=155")

HTTP.ExceptionRequest.StatusError: HTTP.ExceptionRequest.StatusError(502, HTTP.Messages.Response:
"""
HTTP/1.1 502 Bad Gateway
Date: Tue, 27 Nov 2018 10:52:10 GMT
Server: Apache/2.4.23 (Linux/SUSE)
Vary: accept-language,accept-charset
Accept-Ranges: bytes
Transfer-Encoding: chunked
Content-Type: text/html; charset=utf-8
Content-Language: en

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"
  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">
<head>
<title>Bad Gateway!</title>
<link rev="made" href="mailto:%5bno%20address%20given%5d" />
<style type="text/css"><!--/*--><![CDATA[/*><!--*/ 
    body { color: #000000; background-color: #FFFFFF; }
    a:link { color: #0000CC; }
    p, address {margin-left: 3em;}
    span {font-size: smaller;}
/*]]>*/--></style>
</head>

<body>
<h1>Bad Gateway!</h1>
<p>


    The proxy server received an invalid
    response from an up

## Job template definitions

Get the current definitions, filter them and map them to a tuple of new templates (but don't actually sumbit them yet to OpenQA).

In [ ]:
jobs = OpenQA.get_json(OpenQA.osd, "job_templates?group_id=155")["JobTemplates"]
jobs[1]

In [51]:
jobs = OpenQA.get_json(OpenQA.osd, "job_templates")["JobTemplates"]
jobs[1]

Dict{String,Any} with 6 entries:
  "group_name" => "x-released SLE 11 SP4 4.Test Development"
  "prio"       => 50
  "id"         => 6613
  "test_suite" => Dict{String,Any}("name"=>"autoinst","id"=>1650)
  "machine"    => Dict{String,Any}("name"=>"ppc64","id"=>64)
  "product"    => Dict{String,Any}("arch"=>"ppc64","flavor"=>"Server-DVD","dist…

In [55]:
spvmjobs = filter(jobs) do j
    j["machine"]["name"] == "ppc64le-pvm"
end

0-element Array{Any,1}

In [53]:
foreach(display, spvmjobs)

Dict{String,Any} with 6 entries:
  "group_name" => "Functional: Server"
  "prio"       => 30
  "id"         => 14621
  "test_suite" => Dict{String,Any}("name"=>"textmode","id"=>1678)
  "machine"    => Dict{String,Any}("name"=>"ppc64le-spvm","id"=>115)
  "product"    => Dict{String,Any}("arch"=>"ppc64le","flavor"=>"Server-DVD","di…

Dict{String,Any} with 6 entries:
  "group_name" => "Functional"
  "prio"       => 30
  "id"         => 15876
  "test_suite" => Dict{String,Any}("name"=>"textmode","id"=>1678)
  "machine"    => Dict{String,Any}("name"=>"ppc64le-spvm","id"=>115)
  "product"    => Dict{String,Any}("arch"=>"ppc64le","flavor"=>"Installer-DVD",…

In [54]:
spvmjobs[1]["product"]

Dict{String,Any} with 6 entries:
  "arch"    => "ppc64le"
  "flavor"  => "Server-DVD"
  "distri"  => "sle"
  "id"      => 397
  "group"   => "sle-12-SP4-Server-DVD"
  "version" => "12-SP4"

In [ ]:
nujobs = map(filter(jobs) do j
    startswith(j["test_suite"]["name"], "xfstests")      
end) do j
    (id = j["id"],
     product_id = aprods[j["product"]["arch"]],
     machine_id = j["machine"]["id"],
     test_suite_id = j["test_suite"]["id"],
     name = j["test_suite"]["name"])
end

## Sumbitting changes

Send the changes to OpenQA

In [ ]:
client_script = `openqa-client --host openqa.suse.de --apikey C6C16EEAC002010A --apisecret A411FEDE926A5A9A`

Delete the templates we queried earlier

In [ ]:
fails = []

for j in nujobs
    try
        run(`$client_script job_templates/$(j.id) delete`)
    catch e
        push!(fails, (error = e, job = j))
    end
end

Create new templates from what we queried earlier

In [ ]:
fails = []

for j in nujobs
    try
        run(`$client_script job_templates post product_id=$(j.product_id) machine_id=$(j.machine_id) test_suite_id=$(j.test_suite_id) group_id=116`)
    catch e
        push!(fails, (error = e, job = j))
    end
end

If there were failures then we can try to find out why. Usually it is because some test suite is already being run for a particular product, but in a different group.

In [ ]:
ftemplates = map(fails) do f
    jid = f.job.test_suite_id
    OpenQA.get_json(OpenQA.osd, "job_templates?test_suite_id=$jid")["JobTemplates"]
end

In [ ]:
for f in ftemplates
    println(f[1]["test_suite"]["name"], " => ")
    for ff in f
        if ff["product"]["version"] == "15-SP1"
            println("\t", ff["group_name"])
        end
    end
end
    